In [1]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

from ASCE import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, cracked_moment_inertia
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Concrete Beam Design
## Inputs

### Loads

In [1]:
SW = conc_density * b * h / 12**2 #plf
DL = 15 #psf
LL = 40 #psf

LN = ln / 12 #ft
trib = 15 #ft
D = DL * trib / 12 + SW #plf
L = LL * trib / 12 #plf
w_u = load_combos.LRFD(D, L) #plf

Mu = w_u * LN**2 / 8 / 1000 #kip-ft
Vu = w_u * LN / 2 / 1000 #kips
print(f"Mu = {round(Mu / 12, 2)} lb-ft")
print(f"Vu = {round(Vu, 2)} lbs")2 #in-lbs

SyntaxError: invalid syntax (641078349.py, line 14)

### Materials and Geometry

In [2]:
# Materials
fc = 4000 #psi
fy = 60000 #psi
fyt = 60000 #psi
Es = 29000000 #psi

# Dimensions
b = 14 #in
h = 30 #in
cover_t = 1.5 #in
cover_b = 1.5 #in

In [1]:
# Shear Steel
stirrup_size = "#4"
no_legs = 2
s = 12 #in O.C.

db_tie = rebar.loc[stirrup_size, "Diameter"]
Av = no_legs * rebar.loc[stirrup_size, "Area"]
f"Av = {round(Av, 2)} in^2"

NameError: name 'rebar' is not defined

In [10]:
# Tension Steel
bar = "#10"
Ab = rebar.loc[bar, "Area"]
db = rebar.loc[bar, "Diameter"]
n = 4 #number of bars
layers = 1

d_t = h - cover_b - db_tie - db/2
print(f"d_t = {round(d_t, 3)} in")

As = n * Ab #in^2
As_ft = As * 12 / b
print(f"As = {round(As, 3)} in^2")

d_t = 27.365 in
As = 5.08 in^2


In [11]:
# Compression Steel
bar = "#10"
Ab_c = rebar.loc[bar, "Area"]
db_c = rebar.loc[bar, "Diameter"]
n = 4 #number of bars
layers = 1

As_c = n * Ab_c #compression reinforcement
d_c = cover_t + db_c/2
print(f"d_c = {round(d_c, 3)} in")

print(f"As = {round(As, 3)} in^2")

d_c = 2.135 in
As = 5.08 in^2


## Strength Design

### $\phi$ Mn $\geq$ Mu

In [13]:
include_compression_steel = "No"
if include_compression_steel == "No":
    As_c = 0

print(As_c)

0


#### $\phi$ Factor Calculation

In [14]:
beta_1 = 0.85
a = As * fy / (0.85 * fc * b)
c = beta_1 * a
print(f"a = {round(a, 3)} in, c = {round(c, 3)} in")

eps_ty = fy / Es
eps_cu = 0.003

eps_t = eps_cu * (d_t - c)/c
print(f"eps_t = {round(eps_t, 5)} in/in")

a = 6.403 in, c = 5.443 in
eps_t = 0.01208 in/in


In [15]:
if eps_t >= eps_ty + 0.003:
    phi_b = 0.9
elif eps_t <= eps_ty:
    phi_b = 0.65
else:
    phi_b = 0.65 + 0.35 * (eps_t - eps_ty) / 0.003

f"phi_b = {round(phi_b, 3)}"

'phi_b = 0.9'

#### Flexural Strength

In [1]:
Mn = As * fy * (d_t - a/2) # + As_c * fy * (d_t-d_c)
phiMn = phi_b * Mn

print(f"Mu = {round(Mu / 12 / 1000, 2)} kip-ft")
print(f"phiMn = {round(phiMn / 12 / 1000, 2)} kip-ft")
phiMn > Mu

NameError: name 'As' is not defined

### $\phi$ Vn $\geq$ Vu

#### Shear Strength

In [17]:
Vc = 2 * sqrt(fc) * b * d_t #lbs
f"Vc = {round(Vc, 2)} lbs"

'Vc = 48460.01 lbs'

In [18]:
Vs = Av * fyt * d_t / s #lbs
f"Vs = {round(Vs, 2)} lbs"

'Vs = 54730.0 lbs'

In [19]:
phi_v = 0.75
Vn = Vc + Vs
print(f"Vn = {round(Vn, 2)} lbs")
print(f"phiVn = {round(phi_v * Vn, 2)} lbs")

Vn = 103190.01 lbs
phiVn = 77392.51 lbs


In [20]:
phi_v * Vn >= Vu

True

### $\phi$ Tn $\geq$ Tu

### $\phi$ Pn $\geq$ Pu

## Deflection

### Short-term Deflection

In [21]:
w_a = load_combos.ASD(D, L)

Ma = w_a * LN**2 / 8 * 12 #lb-in
f"Ma = {round(Ma / 12, 2)} lb-ft"

In [22]:
fr = 7.5 * sqrt(fc) #psi
Ig = b * h**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in")
yt = h / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

Ig = 31500.0 in


'Mcr = 83009.79 lb-ft'

In [23]:
# Elastic Modulus
Ec = elastic_modulus(fc=fc) #ksi
print(f"Ec = {round(Ec, 2)} ksi")
# Moment of Inertia
Ie = Ig
Icr = cracked_moment_inertia(b=b, h=h, d=d_t, d_c=d_c, Ec=Ec, As=As, As_c=As_c)
print(f"Icr = {round(Icr, 2)} in^4")
if Ma > 2/3*Mcr:
    Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
    print(f"Ie = {round(Ie, 2)} in")
else:
    print("section uncracked")

#Elastic Deflection
defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * 1000 * Ie)
f"Elastic Deflection = {round(defl_elastic, 3)} in"

Ec = 3834.25 ksi
Icr = 17361.99 in^4


### Long-term Deflection

In [24]:
# Deflection Factor
xi = 2.0 #duration factor
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 2.0'

In [25]:
# Long-Term Deflection
defl_long = defl_elastic * lmbda_dlt #in
f"Long-Term Deflection = {round(defl_long, 3)} in"

In [26]:
# Total Deflection
defl_total = defl_elastic + defl_long
f"Total Deflection = {round(defl_total, 3)} in"

## Minimum Requirements

### Dimension Requirements

#### Minimum Beam Depth

In [27]:
h_min = ln / 16
print(f"h_min (simply supported) = {round(h_min, 2)} in")

h_min = ln / 18.5
print(f"h_min (one end continuous) = {round(h_min, 2)} in")

h_min = ln / 21
print(f"h_min (both ends continuous) = {round(h_min, 2)} in")

h_min = ln / 8
print(f"h_min (cantilever) = {round(h_min, 2)} in")

In [28]:
Vu <= phi_v * (Vc + 8*sqrt(fc) * b * d_t)

True

### Minimum Reinforcement

#### Flexural Reinforcement

In [29]:
As_min = max(3*sqrt(fc)/fy, 300/fy) * b * d_t #in^2

f"As,min = {round(As_min, 3)} in^2"

'As,min = 1.916 in^2'

#### Shear Reinforcement

In [30]:
if Vu > phi_v*sqrt(fc)*b*d_t:
    Av_min = max(0.75*sqrt(fc), 50) * b / fyt #in^2
    print(f"Av,min = {round(Av_min, 3)} in^2")
else:
    Av_min = 0
    print("Shear reinforcement not required")

Av,min = 0.012 in^2


#### Torsional Reinforcement